# 제12장 이미지생성

### BEGAN (Boundary Equilibrium GAN)

* Discriminator로 CAE (Convolutional auto-encoder) 사용

****공통 모듈 읽어들이기****

In [1]:
import os

import numpy as np
from tensorflow.python import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import losses
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Conv2D, Conv2DTranspose, Activation, Flatten, Dense, UpSampling2D, Reshape, Lambda, Input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.preprocessing.image import img_to_array, array_to_img

**이미지를 저장하는 함수**


In [2]:
def save_imgs(path, imgs, rows, cols):
    """이미지를 타일 형태로 저장
    
    Arguments:
        path (str): 저장할 폴더 경로
        imgs (np.array): 저장할 이미지 리스트
        rows (int): 타일의 세로 크기
        cols (int): 타일의 가로 크기
        
    Returns:
        None
    """
    base_width = imgs.shape[1]
    base_height = imgs.shape[2]
    channels = imgs.shape[3]
    output_shape = (
        base_height*rows,
        base_width*cols,
        channels
    )
    buffer = np.zeros(output_shape)
    for row in range(rows):
        for col in range(cols):
            img = imgs[row*cols + col]
            buffer[
                row*base_height:(row + 1)*base_height,
                col*base_width:(col + 1)*base_width
            ] = img
    array_to_img(buffer).save(path)

**예제 코드12.1:이미지 데이터 읽어 들이기**

In [3]:
DATA_DIR = 'data/chap12/'
BATCH_SIZE = 16
IMG_SHAPE = (64, 64, 3)

data_gen = ImageDataGenerator(rescale=1/255.)
train_data_generator = data_gen.flow_from_directory(
    directory=DATA_DIR,
    classes=['faces'],
    class_mode=None,
    batch_size=BATCH_SIZE,
    target_size=IMG_SHAPE[:2]
)

Found 19370 images belonging to 1 classes.


**예제 코드12.2:Encoder 정의**

In [4]:
def build_encoder(input_shape, z_size, n_filters, n_layers):
    """Encoder구축
    
    Arguments:
        input_shape (int): 이미지의 shape
        z_size (int): 특징 공간의 차원 수
        n_filters (int): 파일 수
        
    Returns:
        model (Model): 인코더 모델 
    """
    model = Sequential()
    model.add(Conv2D(n_filters, 3, activation='elu', input_shape=input_shape, padding='same'))
    model.add(Conv2D(n_filters, 3, padding='same'))
    
    for i in range(2, n_layers + 1):
        model.add(Conv2D(i*n_filters, 3, activation='elu', padding='same'))
        model.add(Conv2D(i*n_filters, 3, activation='elu', strides=2, padding='same'))
        
    model.add(Conv2D(n_layers*n_filters, 3, padding='same'))
    model.add(Flatten())
    model.add(Dense(z_size))
    
    return model

**예제 코드12.3:생성자/Decoder 정의**

In [5]:
def build_decoder(output_shape, z_size, n_filters, n_layers):
    """Decoder 구축
    
    Arguments:
        output_shape (np.array): 이미지 shape
        z_size (int): 특징 공간의 차원 수
        n_filters (int): 파일 수
        n_layers (int): 레이어 수
        
    Returns:
        model (Model): 디코더 모델 
    """
    # UpSampling2D로 몇 배로 확대할지 계산
    scale = 2**(n_layers - 1)
    # 합성곱층의 처음 입력 사이즈를 scale로부터 역산
    fc_shape = (
        output_shape[0]//scale,
        output_shape[1]//scale,
        n_filters
    )
    # 완전연결 계층에서 필요한 사이즈를 역산
    fc_size = fc_shape[0]*fc_shape[1]*fc_shape[2]
    
    model = Sequential()
    
    # 완전연결 계층
    model.add(Dense(fc_size, input_shape=(z_size,)))
    model.add(Reshape(fc_shape))
    
    # 합성곱층 반복
    for i in range(n_layers - 1):
        model.add(Conv2D(n_filters, 3, activation='elu', padding='same'))
        model.add(Conv2D(n_filters, 3, activation='elu', padding='same'))
        model.add(UpSampling2D())
        
    # 마지막 층은 UpSampling2D가 불필요
    model.add(Conv2D(n_filters, 3, activation='elu', padding='same'))
    model.add(Conv2D(n_filters, 3, activation='elu', padding='same'))

    # 출력층에서는 3채널로
    model.add(Conv2D(3, 3, padding='same'))
    
    return model

**예제 코드12.4:생성자 정의**

In [6]:
def build_generator(img_shape, z_size, n_filters, n_layers):
    decoder = build_decoder(
        img_shape, z_size, n_filters, n_layers
    )
    return decoder

**예제 코드12.5:구분자 정의**

In [7]:
def build_discriminator(img_shape, z_size, n_filters, n_layers):
    # CAE
    encoder = build_encoder(
        img_shape, z_size, n_filters, n_layers
    )
    decoder = build_decoder(
        img_shape, z_size, n_filters, n_layers
    )
    return keras.models.Sequential((encoder, decoder))

**예제 코드12.6:구분자의 학습용 네트워크**

In [8]:
def build_discriminator_trainer(discriminator):
    img_shape = discriminator.input_shape[1:]
    real_inputs = Input(img_shape)
    fake_inputs = Input(img_shape)
    real_outputs = discriminator(real_inputs)
    fake_outputs = discriminator(fake_inputs)

    return Model(
        inputs=[real_inputs, fake_inputs],
        outputs=[real_outputs, fake_outputs]
    )

**예제 코드12.7:네트워크 구축**

In [9]:
n_filters = 64  #  필터 수
n_layers = 4 # 레이어 수
z_size = 32  #  특징 공간의 차원

generator = build_generator(
    IMG_SHAPE, z_size, n_filters, n_layers
)
discriminator = build_discriminator(
    IMG_SHAPE, z_size, n_filters, n_layers
)
discriminator_trainer = build_discriminator_trainer(
    discriminator
)

generator.summary()
# discriminator.layers[1]은 디코더를 나타냄
discriminator.layers[1].summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4096)              135168    
_________________________________________________________________
reshape (Reshape)            (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d (Conv2D)              (None, 8, 8, 64)          36928     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 64)          36928     
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D) 

**예제 코드12.8:손실 함수 정의**

In [10]:
from tensorflow.python.keras.losses import mean_absolute_error


def build_generator_loss(discriminator):
    # discriminator를 사용해서 손실 함수 정의
    def loss(y_true, y_pred):
        # y_true はダミー
        reconst = discriminator(y_pred)
        return mean_absolute_error(
            reconst,
            y_pred
        )
    return loss

**예제 코드12.9:generator 컴파일**

In [11]:
# 초기 학습률(Generator)
g_lr = 0.0001

generator_loss = build_generator_loss(discriminator)
generator.compile(
    loss=generator_loss,
    optimizer=Adam(g_lr)
)

**예제 코드12.10:구분자 컴파일**

In [12]:
# 초기 학습률(Discriminator)
d_lr = 0.0001

# k_var는 수치(일반 변수)
k_var = 0.0
# k はKeras(TensorFlow) のVariable
k = K.variable(k_var)
discriminator_trainer.compile(
    loss=[
        mean_absolute_error,
        mean_absolute_error
    ],
    loss_weights=[1., -k],
    optimizer=Adam(d_lr)
)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


**예제 코드12.11:수렴 판정용 함수 정의**

In [13]:
def measure(real_loss, fake_loss, gamma):
    return real_loss + np.abs(gamma*real_loss - fake_loss)

**예제 코드12.12*학습 코드**

In [14]:
# k의 갱신에 이용할 파라미터
GAMMA = 0.5
LR_K = 0.001

# 반복 수. 100000～1000000 정도로 지정
TOTAL_STEPS = 100000

# 모델과 확인용 생성 이미지를 저장할 폴더
MODEL_SAVE_DIR = 'began/models'
IMG_SAVE_DIR = 'began/imgs'
# 확인용으로 5x5 개의 이미지를 생성
IMG_SAMPLE_SHAPE = (5, 5)
N_IMG_SAMPLES = np.prod(IMG_SAMPLE_SHAPE)


# 저장할 폴더가 없다면 생성
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
os.makedirs(IMG_SAVE_DIR, exist_ok=True)

# 샘플이미지용 랜덤 시드
sample_seeds = np.random.uniform(
    -1, 1, (N_IMG_SAMPLES, z_size)
)

history = []
logs = []

for step, batch in enumerate(train_data_generator):
    # 샘플 수가 BATCH_SIZE를 만족하지 않으면 스킵
    # 전체 이미지의 개수가 BATCH_SIZE의 배수가 아닌 경우 발생
    if len(batch) < BATCH_SIZE:
        continue
    
    # 학습종료
    if step > TOTAL_STEPS:
        break

    # 임의의 값 생성
    z_g = np.random.uniform(
        -1, 1, (BATCH_SIZE, z_size)
    )
    z_d = np.random.uniform(
        -1, 1, (BATCH_SIZE, z_size)
    )
    
    # 생성 이미지(구분자의 학습에 이용)
    g_pred = generator.predict(z_d)
    
    # 생성자를 1스텝 학습시킨다
    generator.train_on_batch(z_g, batch)
    # 구분자를 1스텝 학습시킨다
    _, real_loss, fake_loss = discriminator_trainer.train_on_batch(
            [batch, g_pred],
            [batch, g_pred]
    )

    # k 를 갱신
    k_var += LR_K*(GAMMA*real_loss - fake_loss)
    K.set_value(k, k_var)
    

    # g_measure 을 계산하기 위한 loss 저장
    history.append({
        'real_loss': real_loss,
        'fake_loss': fake_loss
    })

    # 1000번에 1번씩 로그 표시
    if step%1000 == 0:
        # 과거 1000 번의 measure 의 평균
        measurement = np.mean([
            measure(
                loss['real_loss'],
                loss['fake_loss'],
                GAMMA
            )
            for loss in history[-1000:]
        ])
        
        logs.append({
            'k': K.get_value(k),
            'measure': measurement,
            'real_loss': real_loss,
            'fake_loss': fake_loss
        })
        print(logs[-1])

        # 이미지 저장  
        img_path = '{}/generated_{}.png'.format(
            IMG_SAVE_DIR,
            step
        )
        save_imgs(
            img_path,
            generator.predict(sample_seeds),
            rows=IMG_SAMPLE_SHAPE[0],
            cols=IMG_SAMPLE_SHAPE[1]
        )
        # 최신 모델 저장
        generator.save('{}/generator_{}.hd5'.format(MODEL_SAVE_DIR, step))
        discriminator.save('{}/discriminator_{}.hd5'.format(MODEL_SAVE_DIR, step))

Instructions for updating:
Use tf.cast instead.
{'k': -0.00047289272, 'measure': 1.418678104877472, 'real_loss': 0.9457854, 'fake_loss': 0.9457854}
{'k': -0.07920497, 'measure': 0.2361962212845683, 'real_loss': 0.05136793, 'fake_loss': 0.05136793}
{'k': -0.118591346, 'measure': 0.11823619798198343, 'real_loss': 0.06784205, 'fake_loss': 0.06784205}
{'k': -0.15363362, 'measure': 0.10522857859544456, 'real_loss': 0.033345547, 'fake_loss': 0.033345547}
{'k': -0.17890067, 'measure': 0.0758511664448306, 'real_loss': 0.025826242, 'fake_loss': 0.025826242}
{'k': -0.19959486, 'measure': 0.06212127793114632, 'real_loss': 0.043241024, 'fake_loss': 0.043241024}
{'k': -0.22036536, 'measure': 0.062311517434660346, 'real_loss': 0.047195844, 'fake_loss': 0.047195844}
{'k': -0.23881513, 'measure': 0.055420100613031535, 'real_loss': 0.020237759, 'fake_loss': 0.020237759}
{'k': -0.25456056, 'measure': 0.047266687262803316, 'real_loss': 0.02103718, 'fake_loss': 0.02103718}
{'k': -0.26871055, 'measure': 0.

{'k': -0.52230227, 'measure': 0.004687032556801569, 'real_loss': 0.001878124, 'fake_loss': 0.001878124}
{'k': -0.5238055, 'measure': 0.004512460932775866, 'real_loss': 0.0040942356, 'fake_loss': 0.0040942356}
{'k': -0.5252268, 'measure': 0.0042700398536981085, 'real_loss': 0.0034222677, 'fake_loss': 0.0034222677}
{'k': -0.52672905, 'measure': 0.004511998969770502, 'real_loss': 0.0015298049, 'fake_loss': 0.0015298049}
{'k': -0.5283249, 'measure': 0.004787489141745027, 'real_loss': 0.004154928, 'fake_loss': 0.004154928}
{'k': -0.52981997, 'measure': 0.004491429425426759, 'real_loss': 0.0041177543, 'fake_loss': 0.0041177543}
{'k': -0.5313023, 'measure': 0.004453169354062993, 'real_loss': 0.0068964465, 'fake_loss': 0.0068964465}
{'k': -0.53282726, 'measure': 0.0045852587987319565, 'real_loss': 0.0039624916, 'fake_loss': 0.0039624916}
{'k': -0.5343019, 'measure': 0.00442972721013939, 'real_loss': 0.003947408, 'fake_loss': 0.003947408}
{'k': -0.53568995, 'measure': 0.0041701018090825525, 're